# Libs

In [1]:
import os
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sqlite3
from pymongo import MongoClient
import json
from bson import ObjectId

In [2]:
os.getcwd()

'c:\\Users\\caiquefilipini\\OneDrive\\Área de Trabalho\\Data Science\\Gen AI\\projetos\\chatbot-whatsapp\\notebooks'

In [3]:
df = pd.read_excel('../data/base_clientes_excel.xlsx')

# Conexão MongoDB

In [14]:
def conectar_mongo():
    try:
        client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        client.server_info()  # Gera uma exceção se não conseguir conectar
        print("Conexão com MongoDB bem-sucedida.")
        if client:
            db = client['db_conversas']
            collection = db['clientes_conversas']
        return client, db, collection
    except Exception as e:
        print(f"Erro ao conectar ao MongoDB: {e}")
        return None

In [28]:
client, db, collection = conectar_mongo()

Conexão com MongoDB bem-sucedida.


# Teste SQLite

In [ ]:
base_clientes = pd.read_csv('../data/base_clientes.csv')

In [ ]:
base_clientes.head()

In [ ]:
# Especificar o caminho para o diretório onde o banco de dados está localizado
path = "../data/"

# Especificar o valor do CPF como uma string entre aspas simples
cpf_value = 42006925890
query = f"SELECT * FROM customer_database_table WHERE cpf = '{cpf_value}'"

# Usar a declaração `with` para gerenciar a conexão ao banco de dados SQLite
with sqlite3.connect(os.path.join(path, 'customer_database.sqlite')) as conn:
    # Executar a consulta SQL e ler o resultado em um DataFrame do pandas
    base = pd.read_sql_query(query, conn)

In [ ]:
base.head()

In [ ]:
base["nome"].values

# Teste ordenação lista

In [ ]:
qtd_msg = 1

lista = list(range(qtd_msg))
print(lista)

In [ ]:
qtd_msg = 1

lista = range(qtd_msg)
print(lista)

In [ ]:
qtd_msg = 4
lista = list(range(qtd_msg - 1, -1, -1))
for p, i in enumerate(lista):
    print("posição", i)
    print("indice", p+1)
    

# Teste armazenar MongoDB

Funções para inserir cliente e mensagem

In [6]:
def inserir_cliente():
    cliente = collection.find_one({"cpf": cpf})
    # Inserir novo cliente se não existir
    if not cliente:
        collection.insert_one({
            "cpf": cpf,
            "conversas": [
                {
                    "assunto": assunto,
                    "chats": [
                        {
                            "data_hora_inicio": dt_hr_ini,
                            "mensagens": []
                        }
                    ]
                }
            ]
        })
    else:
        # Inserir nova conversa caso o cliente já existir na base
        assunto_existe = collection.find_one({"cpf": cpf, "conversas.assunto": assunto})
        if not assunto_existe:
            # Adicionar um novo assunto com o chat inicial
            collection.update_one(
                {"cpf": cpf},
                {"$push": {
                    "conversas": {
                        "assunto": assunto,
                        "chats": [
                            {
                                "data_hora_inicio": dt_hr_ini,
                                "mensagens": []
                            }
                        ]
                    }
                }}
            )
        else:
            # Adicionar um novo chat ao assunto existente
            collection.update_one(
                {"cpf": cpf, "conversas.assunto": assunto},
                {"$push": {
                    "conversas.$.chats": {
                        "data_hora_inicio": dt_hr_ini,
                        "mensagens": []
                    }
                }}
            )


In [7]:
def inserir_mensagem(dict_mensagem):
    collection.update_one(
        {"cpf": cpf},
        {"$push": {"conversas.$[conversa].chats.$[chat].mensagens": dict_mensagem}},
        array_filters=[
            {"conversa.assunto": assunto},
            {"chat.data_hora_inicio": dt_hr_ini}
        ]
    )

Criando dados fictícios para 2 clientes

In [8]:
cliente_1_negociacao_1 = {
    "cpf": "34366343878",
    "assunto": "Negociação",
    "dt_hr_ini":"2024-07-04 09:00:00",
}

cliente_1_negociacao_2 = {
    "cpf": "34366343878",
    "assunto": "Reclamação",
    "dt_hr_ini":"2024-07-08 08:30:00",
}

cliente_2_negociacao_1 = {
    "cpf": "72388905850",
    "assunto": "Negociação",
    "dt_hr_ini":"2024-07-04 09:00:00",
}

cliente_2_negociacao_2 = {
    "cpf": "72388905850",
    "assunto": "Negociação",
    "dt_hr_ini":"2024-07-07 10:00:00",
}

cliente_2_boleto = {
    "cpf": "72388905850",
    "assunto": "Boleto",
    "dt_hr_ini":"2024-07-06 15:30:00",
}


In [9]:
mensagens_cliente_1_negociacao_1 = [
    {
        "id": 1,
        "data_hora": "2024-07-04 09:00:00",
        "mensagem_cliente": "Bom dia, meu nome é João e estou entrando em contato porque estou com dificuldades para pagar a dívida que tenho com vocês. Gostaria de saber se existe alguma forma de negociarmos esse valor para facilitar o pagamento.",
        "sugestao_ia": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo. Vamos verificar o que podemos fazer. Uma opção seria parcelar a dívida em termos que caberiam melhor no seu orçamento mensal. Outra possibilidade seria aplicarmos um desconto para pagamento à vista. O que acha dessas opções?"
    },
    {
        "id": 2,
        "data_hora": "2024-07-04 09:05:00",
        "mensagem_cliente": "Agradeço a compreensão, Carlos. No momento, a opção de parcelamento seria a mais viável para mim. Não tenho como cobrir o valor total imediatamente.",
        "sugestao_ia": "Entendo, João. Vamos estruturar um plano de parcelamento então.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Entendo, João. Podemos estruturar um plano de parcelamento então. Que tal dividirmos o total devido em 12 meses? Isso ajudaria a ajustar as parcelas ao seu orçamento?"
    },
    {
        "id": 3,
        "data_hora": "2024-07-04 09:10:00",
        "mensagem_cliente": "Isso soa bem, Carlos. As parcelas ficariam de quanto, aproximadamente?",
        "sugestao_ia": "Vou calcular aqui rapidamente.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Vou calcular aqui rapidamente. Considerando o total devido, com o parcelamento em 12 meses, as parcelas ficariam em torno de X reais por mês. Isso seria confortável para você?"
    },
    {
        "id": 4,
        "data_hora": "2024-07-04 09:15:00",
        "mensagem_cliente": "Sim, isso eu consigo fazer. Podemos proceder com esse plano então. Como faço para formalizar esse acordo?",
        "sugestao_ia": "Ótimo, João! Vou enviar agora mesmo para seu e-mail um acordo formal.",
        "rating_sugestao_ia": 5,
        "resposta_final_operador": "Ótimo, João! Vou enviar agora mesmo para seu e-mail um acordo formal com os detalhes do parcelamento e as instruções para assinatura. Assim que receber sua confirmação e assinatura, seu novo plano estará ativo."
    }
]

In [10]:
mensagens_cliente_1_negociacao_2 = [
    {
        "id": 1,
        "data_hora": "2024-07-08 08:30:00",
        "mensagem_cliente": "Olá, sou o João. Estou enfrentando algumas dificuldades financeiras adicionais e gostaria de discutir a possibilidade de uma pausa nos pagamentos da dívida por alguns meses. Isso é possível?",
        "sugestao_ia": "Olá, João! Vamos ver o que podemos fazer.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Olá, João! Sinto muito por ouvir isso. Podemos certamente considerar uma pausa no pagamento. Uma opção seria oferecer uma moratória de três meses sem cobrança de juros durante esse período. Isso ajudaria?"
    },
    {
        "id": 2,
        "data_hora": "2024-07-08 08:35:00",
        "mensagem_cliente": "Isso seria de grande ajuda. Há alguma condição especial ou penalidade após esse período?",
        "sugestao_ia": "Vamos verificar as condições para você, João.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Não haverá penalidades, João. Após a pausa de três meses, você retomará os pagamentos como planejado anteriormente, sem alterações no valor ou acréscimos de juros. Isso soa aceitável para você?"
    },
    {
        "id": 3,
        "data_hora": "2024-07-08 08:40:00",
        "mensagem_cliente": "Sim, isso parece justo. Vamos prosseguir com essa pausa. O que preciso fazer para formalizar essa alteração?",
        "sugestao_ia": "Preparando a documentação necessária para você.",
        "rating_sugestao_ia": 5,
        "resposta_final_operador": "Ótimo, João! Vou preparar a documentação necessária e enviar para o seu e-mail para assinatura. Assim que receber sua assinatura, a pausa entrará em vigor. Agradeço a confiança em nos procurar para resolver esta situação."
    }
]

In [11]:
mensagens_cliente_2_negociacao_1 = [
    {
        "id": 1,
        "data_hora": "2024-07-04 14:00:00",
        "mensagem_cliente": "Olá, aqui é a Ana. Estou passando por um momento financeiro difícil e gostaria de saber se existe alguma possibilidade de renegociarmos a minha dívida para facilitar o pagamento.",
        "sugestao_ia": "Olá, Ana! Estou aqui para ajudar.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Olá, Ana! Compreendo sua situação e quero ajudá-la. Temos algumas opções, como parcelar a dívida ou oferecer um desconto para pagamento à vista. Qual dessas opções lhe parece mais adequada?"
    },
    {
        "id": 2,
        "data_hora": "2024-07-04 14:05:00",
        "mensagem_cliente": "Obrigada pela rápida resposta. Acredito que o parcelamento seria mais adequado para mim no momento, já que não tenho como pagar tudo de uma vez.",
        "sugestao_ia": "Perfeito, Ana. Vamos ver as condições de parcelamento.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Perfeito, Ana. Podemos parcelar o total devido em várias parcelas. Que tal dividirmos em 24 meses? Isso ajudaria a aliviar seu orçamento mensal?"
    },
    {
        "id": 3,
        "data_hora": "2024-07-04 14:10:00",
        "mensagem_cliente": "24 meses seria ótimo, mas quais seriam os valores das parcelas?",
        "sugestao_ia": "Vou calcular os valores para você.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Com base no total devido, o parcelamento em 24 meses resultaria em parcelas de aproximadamente Y reais cada. Isso se ajustaria ao seu orçamento?"
    },
    {
        "id": 4,
        "data_hora": "2024-07-04 14:15:00",
        "mensagem_cliente": "Sim, esse valor é algo que posso gerenciar. Como podemos formalizar esse acordo?",
        "sugestao_ia": "Vou preparar o acordo e enviar para o seu e-mail.",
        "rating_sugestao_ia": 5,
        "resposta_final_operador": "Ótimo, Ana! Vou preparar o acordo e enviar para o seu e-mail com todos os detalhes e as instruções para a assinatura eletrônica. Assim que receber sua confirmação, o novo plano estará ativo."
    }
]

In [12]:
mensagens_cliente_2_negociacao_2 = [
    {
        "id": 1,
        "data_hora": "2024-07-07 10:00:00",
        "mensagem_cliente": "Olá, aqui é a Ana novamente. Após revisar meu orçamento, percebi que as parcelas ainda estão um pouco altas para a minha capacidade atual de pagamento. Há alguma forma de reduzirmos ainda mais o valor mensal?",
        "sugestao_ia": "Olá, Ana! Vamos encontrar uma solução juntos.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Olá, Ana! Obrigado por entrar em contato. Podemos explorar algumas opções para aliviar seu orçamento. Uma possibilidade seria estender o prazo do parcelamento para reduzir o valor mensal. Que tal estendermos para 36 meses?"
    },
    {
        "id": 2,
        "data_hora": "2024-07-07 10:05:00",
        "mensagem_cliente": "Estender o prazo pode ser uma boa ideia, mas isso aumentaria muito os juros? Estou tentando evitar mais despesas a longo prazo.",
        "sugestao_ia": "Vamos calcular os juros e ver como fica.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Compreendo sua preocupação, Ana. Se estendermos para 36 meses, haverá um pequeno aumento na taxa de juros, mas ainda assim o valor mensal seria significativamente menor. Posso fazer uma simulação e enviar-lhe os detalhes para que você possa decidir?"
    },
    {
        "id": 3,
        "data_hora": "2024-07-07 10:10:00",
        "mensagem_cliente": "Por favor, faça a simulação. Quero ver todos os números antes de tomar uma decisão.",
        "sugestao_ia": "Simulação em andamento, Ana.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Perfeito, Ana! Estou enviando para o seu e-mail uma simulação detalhada com os novos valores mensais e o total de juros. Assim que revisar, me avise se deseja prosseguir com esta nova condição ou se podemos buscar outra alternativa."
    }
]

In [13]:
mensagens_cliente_2_boleto = [
    {
        "id": 1,
        "data_hora": "2024-07-05 10:00:00",
        "mensagem_cliente": "Bom dia! Sou a Ana, e eu não recebi o boleto para pagamento da parcela deste mês. Vocês poderiam me enviar novamente?",
        "sugestao_ia": "Bom dia, Ana! Claro, vamos verificar isso para você.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Bom dia, Ana! Peço desculpas pelo transtorno. Vou verificar no sistema e encaminhar o boleto para o seu e-mail imediatamente. Pode me confirmar se o endereço de e-mail para envio continua sendo ana@example.com?"
    },
    {
        "id": 2,
        "data_hora": "2024-07-05 10:02:00",
        "mensagem_cliente": "Isso mesmo, o e-mail está correto. Obrigada por verificar tão rápido!",
        "sugestao_ia": "Nada que agradecer, Ana! Vou enviar agora.",
        "rating_sugestao_ia": 4,
        "resposta_final_operador": "Perfeito, Ana! Já enviei o boleto para o seu e-mail. Você deve recebê-lo dentro de alguns minutos. Por favor, verifique também a sua caixa de spam, caso não encontre na caixa de entrada."
    },
    {
        "id": 3,
        "data_hora": "2024-07-05 10:05:00",
        "mensagem_cliente": "Recebi o boleto, muito obrigada pela ajuda rápida! Aproveitando, há algum desconto para pagamento antecipado?",
        "sugestao_ia": "Que bom que deu tudo certo, Ana! Vou verificar sobre o desconto.",
        "rating_sugestao_ia": 3,
        "resposta_final_operador": "Que bom que recebeu, Ana! Sobre o pagamento antecipado, oferecemos um desconto de 5% para pagamentos realizados até 5 dias antes do vencimento. Se interessar, posso recalcular o boleto com o desconto."
    }
]

Inserindo as informações no banco de dados

In [14]:
collection.delete_many({}) # limpar base antes

DeleteResult({'n': 4, 'ok': 1.0}, acknowledged=True)

In [15]:
lista_clientes = [
    cliente_1_negociacao_1,
    cliente_1_negociacao_2,
    cliente_2_negociacao_1,
    cliente_2_negociacao_2,
    cliente_2_boleto
]

In [16]:
lista_mensagens = [
    mensagens_cliente_1_negociacao_1,
    mensagens_cliente_1_negociacao_2,
    mensagens_cliente_2_negociacao_1,
    mensagens_cliente_2_negociacao_2,
    mensagens_cliente_2_boleto
]

In [17]:
for c, m in zip(lista_clientes, lista_mensagens):
    cpf = c["cpf"]
    assunto = c["assunto"]
    dt_hr_ini = c["dt_hr_ini"]
    inserir_cliente()
    for i in m:
        inserir_mensagem(i)

Função para visualizar os dados

In [18]:
# Função personalizada para serializar ObjectId
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return json.JSONEncoder.default(self, obj)

In [19]:
# Consultar todos os documentos na coleção
def consultar_todos():
    documentos = collection.find()
    for doc in documentos:
        print(json.dumps(doc, indent=4, ensure_ascii=False, cls=JSONEncoder))

# Consultar documentos com uma condição
def consultar_condicao(cpf):
    documento = collection.find_one({"cpf": cpf})
    if documento:
        print(json.dumps(documento, indent=4, ensure_ascii=False, cls=JSONEncoder))
    else:
        print(f"Nenhum documento encontrado para o CPF {cpf}")

In [20]:
consultar_todos()
# consultar_condicao(72388905850)

{
    "_id": "669a38619a12d80e0e22a0b2",
    "cpf": "34366343878",
    "conversas": [
        {
            "assunto": "Negociação",
            "chats": [
                {
                    "data_hora_inicio": "2024-07-04 09:00:00",
                    "mensagens": [
                        {
                            "id": 1,
                            "data_hora": "2024-07-04 09:00:00",
                            "mensagem_cliente": "Bom dia, meu nome é João e estou entrando em contato porque estou com dificuldades para pagar a dívida que tenho com vocês. Gostaria de saber se existe alguma forma de negociarmos esse valor para facilitar o pagamento.",
                            "sugestao_ia": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo.",
                            "rating_sugestao_ia": 3,
                            "resposta_final_operador": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo. Vamos verificar o que podemos fazer. Uma opção seria par

In [21]:
consultar_condicao('21573531600')

Nenhum documento encontrado para o CPF 21573531600


Validado - OK!

# Teste ler conversa whatsapp

In [226]:
def conectar_mongo():
    try:
        client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        client.server_info()  # Gera uma exceção se não conseguir conectar
        print("Conexão com MongoDB bem-sucedida.")
        if client:
            db = client['whatsapp']
            collection = db['messages']
            telefones_teste = db["matriculas"]
        return client, db, collection, telefones_teste
    except Exception as e:
        print(f"Erro ao conectar ao MongoDB: {e}")
        return None

In [227]:
client, db, collection, telefones_teste = conectar_mongo()

Conexão com MongoDB bem-sucedida.


In [245]:
collection.delete_many({})
telefones_teste.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [239]:
documentos = collection.find()

In [240]:
telefones = telefones_teste.find()

In [241]:
for doc in documentos:
    print(json.dumps(doc, default=str, indent=4))

{
    "_id": "66cf08cdd015bc8300b53432",
    "numero": "5511985926125",
    "cpf": 0,
    "data_nascimento": "",
    "primeiro_nome": "",
    "primeira_mensagem": 0,
    "autenticar_matricula": 0,
    "escolher_cliente": 1,
    "comecar_negociacao": 0,
    "autenticar_cpf": 0,
    "autenticar_data_nascimento": 0,
    "inicio_chatbot": 0,
    "reportar_problema": 0,
    "nome_usuario": "Ca\u00edque",
    "chat": []
}


In [243]:
for tel in telefones:
    print(json.dumps(tel, default=str, indent=4))

In [124]:
cpf_informado = 85044167398

In [125]:
base_clientes = pd.read_excel("../data/base_clientes_excel.xlsx", engine="openpyxl")

In [134]:
dados_cliente = base_clientes.drop("dt_ref", axis=1).query("cpf == @cpf_informado").to_dict(orient="records")[0]

In [137]:
dados_cliente["data_nascimento"] = dados_cliente["data_nascimento"].strftime("%Y-%m-%d")

In [138]:
dados_cliente

{'cpf': 85044167398,
 'nome': 'Marina Moraes',
 'segmento': 'Especial',
 'genero': 'Feminino',
 'prob_rolagem': 'Alta',
 'data_nascimento': '1979-11-08',
 'produto': 'Cheque Especial',
 'nro_contrato': 216507078,
 'vlr_divida': 6468.61,
 'dias_atraso': 65}

In [146]:
from pymongo import MongoClient
from bson import ObjectId

client.drop_database("nome_do_banco_de_dados")

# Conexão com o MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['nome_do_banco_de_dados']  # Substitua pelo nome do seu banco de dados
collection = db['nome_da_colecao']     # Substitua pelo nome da sua coleção

# Id do documento onde você quer adicionar o array
document_id = ObjectId()  # Gera um novo ObjectId
new_document = {
    '_id': document_id,
    'conversas': []  # Inicializa uma lista vazia para armazenar mensagens e respostas
}

# document_id = 'algum_id_especifico'

# Inserir o novo documento na coleção
result = collection.insert_one(new_document)
print("Documento inserido com ID:", result.inserted_id)

# Criar os dicionários para mensagem e resposta
mensagem_recebida = {
    "mensagem": "mensagem_recebida",
    "data_hora": "data_hora_recebida",
    "tipo": "recebida"
}

mensagem_resposta = {
    "mensagem": "resposta_enviada",
    "data_hora": "data_hora_enviada",
    "tipo": "enviada"
}

# mensagem = {'tipo': 'mensagem', 'conteudo': 'Olá, como posso ajudar?'}
# resposta = {'tipo': 'resposta', 'conteudo': 'Gostaria de saber sobre o produto X.'}

# Apendar os dicionários à lista no documento
result = collection.update_one(
    {'_id': document_id},           # Filtro para encontrar o documento
    {'$push': {'conversas': {'$each': [mensagem_recebida, mensagem_resposta]}}}  # Adiciona ambos os dicionários
)

print("Documentos modificados:", result.modified_count)

# Buscar o documento
documento = collection.find_one({'_id': document_id})

# Imprimir o documento como JSON
print(json.dumps(documento, default=str, indent=4))

Documento inserido com ID: 66ce735ab407b35264c2b843
Documentos modificados: 1
{
    "_id": "66ce735ab407b35264c2b843",
    "conversas": [
        {
            "mensagem": "mensagem_recebida",
            "data_hora": "data_hora_recebida",
            "tipo": "recebida"
        },
        {
            "mensagem": "resposta_enviada",
            "data_hora": "data_hora_enviada",
            "tipo": "enviada"
        }
    ]
}


In [148]:
[i for i in documento["conversas"]]

[{'mensagem': 'mensagem_recebida',
  'data_hora': 'data_hora_recebida',
  'tipo': 'recebida'},
 {'mensagem': 'resposta_enviada',
  'data_hora': 'data_hora_enviada',
  'tipo': 'enviada'}]

In [145]:
document_id = ObjectId()
document_id

ObjectId('66ce72e7b407b35264c2b841')

In [152]:
cpf_informado = 34366343878

In [163]:
ofertas = pd.read_excel("../data/ofertas.xlsx", engine="openpyxl").query("CPF == @cpf_informado") #.to_dict(orient="records")[0]

In [177]:
ofertas = pd.read_excel("../data/ofertas.xlsx", engine="openpyxl").query("CPF == @cpf_informado") #.to_dict(orient="records")[0]
ofertas.drop("CPF", axis=1, inplace=True)

cols_a_vista = [
    "Valor de Entrada ou à Vista",
    "Desconto Concedido"
]

cols_parcelado = [
    "Valor de Entrada ou à Vista",
    "Quantidade de Parcelas",
    "Valor da Parcela",
    "Taxa de Juros (am)",
    "Taxa CET",
    "Desconto Concedido"
]

ofertas_a_vista = ofertas[ofertas['Tipo de Oferta'] == 'À Vista'][cols_a_vista].rename(columns={"Valor de Entrada ou à Vista": "valor", "Desconto Concedido": "desconto"}).to_dict(orient='records')
ofertas_parcelado = ofertas[ofertas['Tipo de Oferta'] == 'Parcelado'][cols_parcelado].rename(columns={"Valor de Entrada ou à Vista": "valor", "Quantidade de Parcelas": "parcelas", "Valor da Parcela": "valor_parcela", "Taxa de Juros (am)": "taxa_juros", "Taxa CET": "taxa_cet", "Desconto Concedido": "desconto"}).to_dict(orient='records')

# Agrupar os resultados em um dicionário JSON
relacao_ofertas = {
    'Ofertas À Vista': ofertas_a_vista,
    'Ofertas Parcelado': ofertas_parcelado
}

In [178]:
resultado_json

{'Ofertas À Vista': [{'valor': 'R$ 2978,29', 'desconto': '20,0%'},
  {'valor': 'R$ 3350,57', 'desconto': '10,0%'}],
 'Ofertas Parcelado': [{'valor': 'R$ 1116,86',
   'parcelas': 12,
   'valor_parcela': 'R$ 231,54',
   'taxa_juros': '1,0%',
   'taxa_cet': '1,0%',
   'desconto': '0,0%'},
  {'valor': 'R$ 1116,86',
   'parcelas': 24,
   'valor_parcela': 'R$ 130,10',
   'taxa_juros': '1,5%',
   'taxa_cet': '1,5%',
   'desconto': '0,0%'},
  {'valor': 'R$ 1116,86',
   'parcelas': 36,
   'valor_parcela': 'R$ 102,24',
   'taxa_juros': '2,0%',
   'taxa_cet': '2,0%',
   'desconto': '0,0%'}]}

In [121]:
# Função _carrega_prompt
def _carrega_prompt(dados_cliente, ofertas_disponiveis, historico_mensagens):
    with open("prompt_negociacao.txt", "r", encoding="utf-8") as file:
        return file.read().format(
            dados_cliente=dados_cliente, 
            ofertas_disponiveis=ofertas_disponiveis, 
            historico_mensagens=historico_mensagens
        )

In [113]:
dados_cliente = "CLIENTE"
ofertas_disponiveis = "OFERTAS"
historico_mensagens = "HISTORICO"

In [114]:
prompt = _carrega_prompt(dados_cliente, ofertas_disponiveis, historico_mensagens)

In [116]:
prompt

Um abraço. Sander!
</script_encerramento_nao_negocio>

[VARIAVEIS]

CLIENTE

OFERTAS


# Teste leitura dados MongoDB

In [64]:
# Percorre os documentos para extrair as informações necessárias
documentos = collection.find()
conversas_lista = []
for documento in documentos:
    cpf = documento["cpf"]
    for conversa in documento["conversas"]:
        assunto = conversa["assunto"]
        for chat in conversa["chats"]:
            data_hora_inicio = chat["data_hora_inicio"]
            conversas_lista.append(f"{data_hora_inicio} - {cpf} - {assunto}")

In [65]:
# Ordena a lista por data
conversas_lista.sort()

# Exibe a lista ordenada
for conversa in conversas_lista:
    print(conversa)

2024-07-04 09:00:00 - 34366343878 - Negociação
2024-07-04 09:00:00 - 72388905850 - Negociação
2024-07-06 15:30:00 - 72388905850 - Boleto
2024-07-07 10:00:00 - 72388905850 - Negociação
2024-07-08 08:30:00 - 34366343878 - Reclamação
2024-07-16 19:43 - 02088667105 - Reclamação


In [ ]:
# Percorre o dicionário para extrair as informações necessárias
for pessoa in dicionario:
    cpf = pessoa["cpf"]
    for conversa in pessoa["conversas"]:
        assunto = conversa["assunto"]
        for chat in conversa["chats"]:
            data_hora_inicio = chat["data_hora_inicio"]
            conversas_lista.append(f"{data_hora_inicio} - {cpf} - {assunto}")

# Ordena a lista por data
conversas_lista.sort()

# Exibe a lista ordenada
for conversa in conversas_lista:
    print(conversa)

In [ ]:
lista = ["2024-06-27 - 42006925890 - Reclamação", "2024-06-27 - 42006925890 - Boleto"]

In [ ]:
lista[:]

In [ ]:
lista2 = []
for i in lista:
    lista2.append(i.split(" - "))

In [ ]:
lista2

Criando conversas fictícias para testar a sidebar

In [ ]:
posicoes_lista = []

# Teste leitura política

In [ ]:
def _calcula_ciclo(dias_atraso):
    ciclos = [
        (30, "Ciclo 1"),
        (60, "Ciclo 2"),
        (90, "Ciclo 3"),
        (120, "Ciclo 4"),
        (150, "Ciclo 5"),
    ]
    for limite, ciclo in ciclos:
        if dias_atraso <= limite:
            return ciclo
    return "Ciclo 6"

In [ ]:
def _carregar_politica(dias_atraso, prob_rolagem):
    path = "../data/"
    file = "politica.xlsx"
    ciclo = _calcula_ciclo(dias_atraso)

    dados_politica = pd.read_excel(os.path.join(path, file))
    dados_politica = dados_politica[(dados_politica["ciclo"] == ciclo) & (dados_politica["prob_rolagem"] == prob_rolagem)]
    dados_politica = dados_politica.to_dict(orient="records")[0]
    return dados_politica

In [ ]:
_carregar_politica(dias_atraso=70, prob_rolagem="Baixa")

# Teste carregar dados cliente

In [25]:
def _carregar_dados_cliente(cpf):
        path = "../data/"
        file = "base_clientes_excel.xlsx"
        consulta_cliente = pd.read_excel(os.path.join(path, file))
        consulta_cliente = consulta_cliente[consulta_cliente["cpf"] == cpf]

        dados_cliente = {}
        if consulta_cliente.shape[0] > 0:
            dados_cliente = {
                "nome": consulta_cliente["nome"][0],
                "segmento": consulta_cliente["segmento"][0],
                "genero": consulta_cliente["genero"][0],
                "prob_rolagem": consulta_cliente["prob_rolagem"][0],
                "data_nascimento": consulta_cliente["data_nascimento"][0].strftime("%Y-%m-%d"),
                "idade": relativedelta(datetime.today(), consulta_cliente["data_nascimento"][0]).years,
                "produto": consulta_cliente["produto"][0],
                "nro_contrato": consulta_cliente["nro_contrato"][0],
                "vlr_divida": consulta_cliente["vlr_divida"][0],
                "dias_atraso": consulta_cliente["dias_atraso"][0]
            }
        return dados_cliente

In [28]:
path = "../data/"
file = "base_clientes_excel.xlsx"
consulta_cliente = pd.read_excel(os.path.join(path, file))

In [30]:
consulta_cliente.dtypes

dt_ref             datetime64[ns]
cpf                         int64
nome                       object
segmento                   object
genero                     object
prob_rolagem               object
data_nascimento    datetime64[ns]
produto                    object
nro_contrato                int64
vlr_divida                float64
dias_atraso                 int64
dtype: object

In [26]:
_carregar_dados_cliente(cpf=34366343878)

{'nome': 'Juan Moura',
 'segmento': 'Private',
 'genero': 'Masculino',
 'prob_rolagem': 'Alta',
 'data_nascimento': '1998-01-05',
 'idade': 26,
 'produto': 'Cartao de Credito',
 'nro_contrato': 831783229,
 'vlr_divida': 3722.86,
 'dias_atraso': 78}

# Teste carregar histórico conversa

In [ ]:
cliente_1_negociacao_1 = {
    "cpf": 34366343878,
    "assunto": "Negociação",
    "dt_hr_ini":"2024-07-04 09:00:00",
}

In [ ]:
cpf = cliente_1_negociacao_1["cpf"]
assunto = cliente_1_negociacao_1["assunto"]
dt_hr_ini = cliente_1_negociacao_1["dt_hr_ini"]

In [ ]:
print(cpf, assunto, dt_hr_ini)

In [ ]:
def _carregar_dados_conversa(asc=False):
    dados_conversa = collection.aggregate([
        {"$match": {"numero": telefone}},
        {"$match": {"cpf": cpf}},
        {"$unwind": "$conversas.chats"},
        {"$project": {
            "_id": 0,
            "chat": "$conversas.chats"
        }}
    ])

    dados_conversa = list(dados_conversa)[0]["chat"]["mensagens"]
    return dados_conversa

In [ ]:
cpf = 72388905850
assunto = "Boleto"
dt_hr_ini = "2024-07-06 15:30:00"

In [ ]:
_carregar_dados_conversa()

In [ ]:
# if st.session.carregar_historico:
#     dados_conversa = _carregar_dados_conversa()
#     st.session.carregar_historico = False

dados_conversa = _carregar_dados_conversa()

In [ ]:
dados_conversa

Adicionou mais uma mensagem? Appenda no dados conversa... Não precisa ficar carregando toda vez...

Situações:

1. Novo chat (não precisa carregar)
2. Seleciona chat existente (precisa carregar)
3. Adiciona primeira mensagem no chat novo (não precisa carregar, só appendar)
4. Adiciona mais uma mensagem no chat existente (não precisa carregar, só appendar)

Conclusão
1. Inicia com uma variável dados_conversa = [] ou carregar_chat_historico = False.
    - Se algum botão de chat histórico for selecionado, carregar_chat_historico = True.
    - Se carregar_chat_historico = True, função carregar executa e carregar_chat_historico = False
2. Quando o usuário inserir uma nova mensagem, dados_conversa.append(dict_nova_mensagem)
3. Exibir histórico se len(dados_conversa) > 0
4. Somente exibir na sidebar clientes que possuem pelo menos 1 mensagem registrada.

In [ ]:
dados_conversa

In [ ]:
keys_historico_bot = ['id', 'data_hora', 'mensagem_cliente', "resposta_final_operador"]
historico_mensagens = []
for i in dados_conversa:
    novo_dict_conversa = {k: i[k] for k in keys_historico_bot}
    historico_mensagens.append(novo_dict_conversa)

In [ ]:
historico_mensagens

In [ ]:
def exibir_historico_conversa(dados_conversa, asc=False):
    dados_conversa_exibir = dados_conversa.copy()
    if not asc:
        dados_conversa_exibir.reverse()
    if len(dados_conversa) > 0:
        print("---")
        print("### Histórico da Conversa")
        print("---")
        # st.button(classificar)
        for conversa in dados_conversa_exibir:
            print(f"ID: {conversa['id']} - Data e Hora: {conversa['data_hora']}")
            print(f"Mensagem do cliente: {conversa['mensagem_cliente']}")
            print(f"Sugestão da IA: {conversa['sugestao_ia']}  (Nota {conversa['rating_sugestao_ia']})")
            print(f"Resposta enviada ao cliente: {conversa['resposta_final_operador']}")
            print("---")

appenda no final se asc = True, senão insert 0. melhor solução é colocar o param asc dentro de exibir. e sempre usar a função de appendar na conversa.

In [ ]:
# Definindo a lista inicial
minha_lista = [2, 3, 4]

# Elemento que você quer inserir na primeira posição
elemento = 1

# Inserindo o elemento na primeira posição
minha_lista.insert(0, elemento)

# Mostrando a lista após a inserção
print(minha_lista)

In [ ]:
minha_lista.append(5)

In [ ]:
minha_lista

In [ ]:
exibir_historico_conversa(dados_conversa, asc=False)

# Teste leitura armaz. novo chat

In [4]:
from save_data import SaveData

In [ ]:
SaveData().inserir_cliente()

In [ ]:
cpf = "72388905850"
assunto = "Negociação"
data_hora_inicio = "2024-07-04 09:00:00"

In [ ]:
dict_mensagem = {
    "id":5,
    "data_hora":"2024-07-09 - 15:45",
    "mensagem_cliente":"oi",
    "sugestao_ia":"Olá! O que posso fazer por você hoje?",
    "rating_sugestao_ia":"3 – Resposta Razoável",
    "resposta_final_operador":"Olá! O que posso fazer por você hoje?"
}

In [ ]:
def inserir_mensagem(self, dict_mensagem):
    """Insere uma nova mensagem no banco de dados MongoDB."""

    self.collection.update_one(
        {"cpf": cpf},
        {"$push": {"conversas.$[conversa].chats.$[chat].mensagens": dict_mensagem}},
        array_filters=[
            {"conversa.assunto": assunto},
            {"chat.data_hora_inicio": data_hora_inicio}
        ]
    )

In [74]:
consultar_todos()

{
    "_id": "668de07bf72d0abe626ae23e",
    "cpf": "34366343878",
    "conversas": [
        {
            "assunto": "Negociação",
            "chats": [
                {
                    "data_hora_inicio": "2024-07-04 09:00:00",
                    "mensagens": [
                        {
                            "id": 1,
                            "data_hora": "2024-07-04 09:00:00",
                            "mensagem_cliente": "Bom dia, meu nome é João e estou entrando em contato porque estou com dificuldades para pagar a dívida que tenho com vocês. Gostaria de saber se existe alguma forma de negociarmos esse valor para facilitar o pagamento.",
                            "sugestao_ia": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo.",
                            "rating_sugestao_ia": 3,
                            "resposta_final_operador": "Olá, João! Agradeço seu contato e estou aqui para ajudá-lo. Vamos verificar o que podemos fazer. Uma opção seria par

In [119]:
consultar_condicao("31472259920")


{
    "_id": "668de446284af247130f2949",
    "cpf": "31472259920",
    "conversas": [
        {
            "assunto": "Boleto",
            "chats": [
                {
                    "data_hora_inicio": "2024-07-09 22:30",
                    "mensagens": []
                }
            ]
        }
    ]
}
